<h1 style="text-align: center;">TF2202 Komputasi Rekayasa - Sistem Persamaan Linear</h1>

<h2 style="text-align: center;">Fadjar Fathurrahman</h2>

# Setup

In [1]:
import numpy as np

In [2]:
import IPython.display
from IPython.display import Markdown

Some functions for pretty-printing matrix in notebook:

In [34]:
def colvec_to_tex(v):
    assert len(v.shape) == 1
    lines = str(v).replace('[', '').replace(']', '').split()
    rv = r'\begin{bmatrix}'
    for l in lines:
        rv = rv + l + r" \\"
    rv +=  r'\end{bmatrix}'
    return rv

# Sumber:
# https://stackoverflow.com/questions/17129290/numpy-2d-and-1d-array-to-latex-bmatrix
def matrix_to_tex(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

def show_matrix(a):
    IPython.display.display(Markdown(matrix_to_tex(a)))
    
def show_col_vector(a):
    IPython.display.display(Markdown(colvec_to_tex(a)))

# Pendahuluan

Pada catatan ini kita akan fokus pada metode untuk mencari solusi dari sistem persamaan linear yang dapat dituliskan dalam bentuk matriks sebagai berikut:
$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$
di mana $A$ dan $b$ masing-masing diberikan dan tugas kita adalah mencari $\mathbf{x}$.

Sebelum membahas mengenai metode numerik untuk menyelesaikan sistem persamaan linear, kita akan mulai dengan pembahasan mengenai operasi matriks dan vektor dengan dalam Numpy.

## List

List pada Python dapat digunakan untuk menyatakan kompulan angka seperti vektor atau matriks.

In [5]:
my_lst = [
    [4,1,2,3],
    [3,8,1,9],
    [3,4,10,4]
]

Pada kuliah di TPB, Anda mungkin mengenal penggunakan `list` ini pada bahasan mengenai array.

List pada Python sebenarnya memiliki fitur yang lebih luas dibandingkan dengan array pada bahasa pemrograman lain, misalnya C++. Misalnya, tipe data pada pada `list` tidak harus seragam:

In [6]:
my_lst1 = [
    ["aaaa",1,2,3],
    [3,8,1,9],
    [3,4,10,4]
]

In [7]:
type(my_lst)

list

In [8]:
my_lst[1][2]

1

## Matrix vs ndarray

Untuk kebutuhan *technical/scientific computing* pada Python, kita biasanya menggunakan pustaka NumPy yang menyediakan array multidimensional dengan tipe data seragam. Array pada NumPy lebih teroptimasi dibandingkan dengan `list` sehingga cocok untuk digunakan pada perhitungan numerik yang berat atau kompleks.

`ndarray` adalah tipe array yang paling penting pada Numpy. Untuk merepresentasikan matriks kita dapat menggunakan `ndarray` dengan menggunakan `ndarray` dengan dua dimensi. Untuk merepresentasikan vektor (baris atau kolom) kita dapat menggunakan `ndarray` dengan satu dimensi.

Contoh untuk matriks:
$$
A = \begin{bmatrix}
4 & 1 & 2 & 3 \\
3 & 8 & 1 & 9 \\
3 & 4 & 10 & 4
\end{bmatrix}
$$

Dengan menggunakan `list`, kita dapat mendefinisikan matrix $A$ sebagai berikut

In [18]:
A_list = [
  [4,1,2,3],
  [3,8,1,9],
  [3,4,10,4]  
]
A_list

[[4, 1, 2, 3], [3, 8, 1, 9], [3, 4, 10, 4]]

Dengan menggunakan `ndarray`, kita dapa mendefinisikan matriks $A$ dengan:

In [20]:
A = np.array([
    [4,1,2,3],
    [3,8,1,9],
    [3,4,10,4]
])
A

array([[ 4,  1,  2,  3],
       [ 3,  8,  1,  9],
       [ 3,  4, 10,  4]])

Alternatif lain:

In [21]:
A = np.array(A_list)

In [107]:
type(A)

numpy.ndarray

Properti `shape` dapat digunakan untuk mengetahu ukuran dari `ndarray`. Dalam hal ini kita akan mendapatkan tupel berisi dua integer, yang masing-masing integer merupakan jumlah baris dan kolom dari matriks $A$

In [13]:
A.shape

(3, 4)

In [14]:
A.diagonal()

array([ 4,  8, 10])

Anda dapat menuliskan sebagai berikut:

In [109]:
Nrow = A.shape[0]
Ncol = A.shape[1]
print("Nrow = %d, Ncol = %d" % (Nrow, Ncol))

Nrow = 3, Ncol = 4


In [110]:
Nrow, Ncol = A.shape
print("Nrow = %d, Ncol = %d" % (Nrow, Ncol))

Nrow = 3, Ncol = 4


Untuk merepresentasikan vektor, kita dapat menggunakan `ndarray` 1d, misalnya:

In [18]:
x = np.array([3,1,6,7])
x

array([3, 1, 6, 7])

In [19]:
type(x)

numpy.ndarray

In [20]:
x.size

4

In [21]:
A.size

12

In [27]:
show_matrix(A)

\begin{bmatrix}
  4 & 1 & 2 & 3\\
  3 & 8 & 1 & 9\\
  3 & 4 & 10 & 4\\
\end{bmatrix}

In [26]:
show_matrix(A.T)

\begin{bmatrix}
  4 & 3 & 3\\
  1 & 8 & 4\\
  2 & 1 & 10\\
  3 & 9 & 4\\
\end{bmatrix}

Properti `shape` dapat digunakan seperti pada `ndarray` dua dimensi. Pada kasus ini akan dikembalikan tupel dengan satu bilangan integer.

In [112]:
x.shape

(4,)

Fungsi `len` juga dapat digunakan dalam kasus `ndarray` 1d untuk mengetahui jumlah elemen pada suatu vektor:

In [113]:
len(x)

4

Fungsi `len` juga dapat diaplikasikan pada `ndarray` dua dimensi, namun fungsi ini akan mengembalikan banyak elemen pada dimensi pertama.

In [114]:
len(A)

3

Untuk mengetahui jumlah kolom, kita juga dapat mencari panjang dari `A[0]` (misalnya):

In [115]:
len(A[0])

4

## Operasi dengan ndarray

In [29]:
A = np.array([
    [4,1,2,3],
    [3,8,1,9],
    [3,4,10,4]
], dtype=np.float64)
A

array([[ 4.,  1.,  2.,  3.],
       [ 3.,  8.,  1.,  9.],
       [ 3.,  4., 10.,  4.]])

Mengakses elemen

In [36]:
A[0,2]

2.0

Menulis nilai ke suatu elemen

In [37]:
A[0,2] = 999.0

In [28]:
A

array([[ 4.,  1.,  2.,  3.],
       [ 3.,  8.,  1.,  9.],
       [ 3.,  4., 10.,  4.]])

Slicing:

In [26]:
A[:,0]

array([4., 3., 3.])

In [27]:
A[:,1]

array([1., 8., 4.])

In [42]:
A[:,1:2]

array([[1.],
       [8.],
       [4.]])

In [35]:
show_col_vector(A[1,:])

\begin{bmatrix}3. \\8. \\1. \\9. \\\end{bmatrix}

In [30]:
show_matrix(A[:,1])

\begin{bmatrix}
  1. & 8. & 4.\\
\end{bmatrix}

 ### Operasi perkalian matriks dan vektor

Untuk menghitung operasi perkalian, misalnya $\mathbf{b} = \mathbf{A}\mathbf{x}$.
Untuk tipe `numpy.ndarray` kita dapat menggunakan fungsi `np.matmul`:

`np.matmul` juga sering digunakan untuk operasi kontraksi tensor (tensor contraction).

In [43]:
A

array([[  4.,   1., 999.,   3.],
       [  3.,   8.,   1.,   9.],
       [  3.,   4.,  10.,   4.]])

In [44]:
x

array([3, 1, 6, 7])

In [45]:
b = np.matmul(A,x)
b

array([6028.,   86.,  101.])

Operator `*` memiliki arti yang berbeda untuk operasi antara dua `ndarray`:

In [46]:
A

array([[  4.,   1., 999.,   3.],
       [  3.,   8.,   1.,   9.],
       [  3.,   4.,  10.,   4.]])

In [50]:
x

array([3, 1, 6, 7])

In [47]:
Ax = A*x
Ax

array([[1.200e+01, 1.000e+00, 5.994e+03, 2.100e+01],
       [9.000e+00, 8.000e+00, 6.000e+00, 6.300e+01],
       [9.000e+00, 4.000e+00, 6.000e+01, 2.800e+01]])

In [49]:
3*A[0]

array([  12.,    3., 2997.,    9.])

In [119]:
print(Ax[:,0]/A[:,0])
print(Ax[:,1]/A[:,1])
print(Ax[:,2]/A[:,2])

[3. 3. 3.]
[1. 1. 1.]
[6. 6. 6.]


Jika ingin menggunakan operator `*` kita dapat menggunakan konstruktor `np.matrix`.

In [66]:
B = np.random.rand(4,4)
B

array([[0.95572449, 0.12716237, 0.31211438, 0.62233433],
       [0.70922067, 0.45498005, 0.6905198 , 0.77237367],
       [0.52556334, 0.72056843, 0.11983184, 0.32861616],
       [0.78915445, 0.6037158 , 0.69502065, 0.57083555]])

In [67]:
AA = np.matrix(A)
BB = np.matrix(B)

In [57]:
AA

matrix([[  4.,   1., 999.,   3.],
        [  3.,   8.,   1.,   9.],
        [  3.,   4.,  10.,   4.]])

In [68]:
BB

matrix([[0.95572449, 0.12716237, 0.31211438, 0.62233433],
        [0.70922067, 0.45498005, 0.6905198 , 0.77237367],
        [0.52556334, 0.72056843, 0.11983184, 0.32861616],
        [0.78915445, 0.6037158 , 0.69502065, 0.57083555]])

In [59]:
type(AA)

numpy.matrix

In [69]:
AA.shape, BB.shape

((3, 4), (4, 4))

Operator `*` dapat digunakan untuk melakukan operasi perkalian matrix antara dua variabel dengan tipe `numpy.matrix`.

In [70]:
AA*BB

matrix([[531.93736167, 722.62263707, 123.73604482, 333.26176524],
        [ 16.16889216,  10.17533812,  12.83551923,  13.51212846],
        [ 14.11630735,  11.82195478,   7.67682332,  10.5260015 ]])

In [71]:
show_matrix(AA*BB)

\begin{bmatrix}
  531.93736167 & 722.62263707 & 123.73604482 & 333.26176524\\
  16.16889216 & 10.17533812 & 12.83551923 & 13.51212846\\
  14.11630735 & 11.82195478 & 7.67682332 & 10.5260015\\
\end{bmatrix}

### Perkalian dot (skalar)

Untuk operasi skalar antara dua vektor kita dapat menggunakan fungsi `np.dot`

In [126]:
y = np.array([2,1,3,6])
y

array([2, 1, 3, 6])

In [127]:
np.dot(y,y)

50

In [128]:
np.dot(x,y)

67

In [129]:
np.dot(x,x)

95

Metode `dot` juga dapat digunakan untuk operasi dot product:

In [130]:
x.dot(x)

95

Operasi `dot` juga dapat digunakan untuk melakukan perkalian antara matriks dengan vektor:

In [131]:
A.dot(x)

array([ 46,  86, 101])

In [132]:
np.matmul(A,x)

array([ 46,  86, 101])

Untuk tipe `numpy.matrix`:

In [133]:
xx = np.matrix(x).transpose()
xx

matrix([[3],
        [1],
        [6],
        [7]])

In [134]:
xx.transpose().dot(xx)[0,0]

95

In [135]:
AA.dot(xx)

matrix([[ 46],
        [ 86],
        [101]])

In [136]:
AA*xx

matrix([[ 46],
        [ 86],
        [101]])

In [139]:
show_matrix(AA*xx)

\begin{bmatrix}
  46\\
  86\\
  101\\
\end{bmatrix}

In [140]:
show_matrix( AA.dot(BB) )

\begin{bmatrix}
  4.11807134 & 3.11530414 & 7.43016495\\
  9.93344831 & 7.75543765 & 17.91842859\\
  10.2711802 & 7.55856248 & 11.56230443\\
\end{bmatrix}

In [142]:
show_matrix( AA*BB )

\begin{bmatrix}
  4.11807134 & 3.11530414 & 7.43016495\\
  9.93344831 & 7.75543765 & 17.91842859\\
  10.2711802 & 7.55856248 & 11.56230443\\
\end{bmatrix}

Pada catatan ini, kita akan menggunakan `matrix`. Jika Anda lebih suka menggunakan `ndarray` langsung, Anda harus melakukan beberapa modifikasi.

Operasi vektorisasi sebisa mungkin akan dihindari agar struktur loop terlihat secara eksplisit.

# Metode Eliminasi Gauss

Dalam metode eliminasi Gauss, matriks $\mathbf{A}$ dan vektor kolom $\mathbf{b}$ akan ditransformasi sedemikian rupa sehingga diperoleh matriks dalam bentuk segitiga atas atau segitiga bawah.

Transformasi yang dilakukan adalah sebagai berikut.
$$
\begin{align*}
A_{ij} \leftarrow A_{ij} - \alpha A_{kj} \\
b_{i} \leftarrow b_{i} - \alpha b_{k}
\end{align*}
$$
Setelah matriks $\mathbf{A}$ direduksi menjadi bentuk segitiga atas, solusi persamaan linear yang dihasilkan dapat dicari dengan menggunakan substitusi mundur (*backward substitution*).
$$
x_{k} = \left(
b_{k} - \sum_{j=k+1}^{N} A_{kj} x_{j}
\right)\frac{1}{A_{kk}}\,\,\,, k = N-1, N-2, \ldots, 1
$$

## Contoh penggunaan metode eliminasi Gauss

Perhatikan sistem persamaan linear berikut ini:

$$
\begin{align*}
x_{1} + x_{2} + x_{3} & = 4 \\
2x_{1} + 3x_{2} + x_{3} & = 9 \\
x_{1} - x_{2} - x_{3} & = -2
\end{align*}
$$

Persamaan di atas dapat diubah dalam bentuk matriks sebagai

$$
\begin{bmatrix}
1 & 1 & 1 \\
2 & 3 & 1 \\
1 & -1 & -1
\end{bmatrix}
\begin{bmatrix}
x_{1} \\
x_{2} \\
x_{3}
\end{bmatrix} =
\begin{bmatrix}
4 \\
9 \\
-2
\end{bmatrix}
$$

In [80]:
A = np.matrix([
    [1, 1, 1],
    [2, 3, 1],
    [1, -1, -1]
], dtype=np.float64)
show_matrix(A)

\begin{bmatrix}
  1. & 1. & 1.\\
  2. & 3. & 1.\\
  1. & -1. & -1.\\
\end{bmatrix}

In [82]:
b = np.matrix([4, 9, -2], dtype=np.float64).transpose()
show_matrix(b)

\begin{bmatrix}
  4.\\
  9.\\
  -2.\\
\end{bmatrix}

Karena kita akan memodifikasi matrix `A` dan `b`, maka kita harus membuat backup (copy) dari nilai asli mereka.

In [83]:
A_orig = np.matrix.copy(A)
b_orig = np.matrix.copy(b)

Dimulai dengan menggunakan elemen matriks $A_{11}$ (atau `A[0,0]` dalam notasi Numpy) sebagai pivot, kita akan melakukan reduksi pada baris kedua dan ketiga.

Kita akan mulai dengan reduksi baris kedua.

In [87]:
show_matrix(A)

\begin{bmatrix}
  1. & 1. & 1.\\
  2. & 3. & 1.\\
  1. & -1. & -1.\\
\end{bmatrix}

In [85]:
show_matrix(A[1,:])

\begin{bmatrix}
  2. & 3. & 1.\\
\end{bmatrix}

In [88]:
alpha = A[1,0]/A[0,0]
alpha

2.0

In [89]:
alpha = A[1,0]/A[0,0]
A[1,:] = A[1,:] - alpha*A[0,:] # baris kedua = baris kedua - alpha*(baris pertama)
b[1] = b[1] - alpha*b[0]

print("A = ")
show_matrix(A)
print("b = ")
show_matrix(b)

A = 


\begin{bmatrix}
  1. & 1. & 1.\\
  0. & 1. & -1.\\
  1. & -1. & -1.\\
\end{bmatrix}

b = 


\begin{bmatrix}
  4.\\
  1.\\
  -2.\\
\end{bmatrix}

Masih menggunakan `A[0,0]` sebagai pivot, kita akan reduksi baris ketiga:

In [90]:
alpha = A[2,0]/A[0,0]
A[2,:] = A[2,:] - alpha*A[0,:]
b[2] = b[2] - alpha*b[0]

print("A = ")
show_matrix(A)
print("b = ")
show_matrix(b)

A = 


\begin{bmatrix}
  1. & 1. & 1.\\
  0. & 1. & -1.\\
  0. & -2. & -2.\\
\end{bmatrix}

b = 


\begin{bmatrix}
  4.\\
  1.\\
  -6.\\
\end{bmatrix}

Setelah menjadikan `A[0,0]` sebagai pivot dan mereduksi baris kedua dan ketiga, kita akan menggunakan `A[1,1]` sebagai pivot dan mereduksi baris ketiga:

In [91]:
alpha = A[2,1]/A[1,1]
A[2,:] = A[2,:] - alpha*A[1,:]
b[2] = b[2] - alpha*b[1]

print("A = ")
show_matrix(A)
print("b = ")
show_matrix(b)

A = 


\begin{bmatrix}
  1. & 1. & 1.\\
  0. & 1. & -1.\\
  0. & 0. & -4.\\
\end{bmatrix}

b = 


\begin{bmatrix}
  4.\\
  1.\\
  -4.\\
\end{bmatrix}

Sekarang `A` telah tereduksi menjadi bentuk matriks segitiga atas. Persamaan yang kita miliki sekarang adalah:

$$
\begin{bmatrix}
1 & 1 & 1 \\
0 & 1 & -1 \\
0 & 0 & -4
\end{bmatrix}
\begin{bmatrix}
x_{1} \\
x_{2} \\
x_{3}
\end{bmatrix} =
\begin{bmatrix}
4 \\
1 \\
-4
\end{bmatrix}
$$

Sistem persamaan linear ini dapat dengan mudah diselesaikan dengan menggunakan substitusi balik: mulai dari mencari $x_3$, kemudian $x_2$, dan akhirnya $x_1$.

In [92]:
N = len(b)
x = np.matrix(np.zeros((N,1)))

x[N-1] = b[N-1]/A[N-1,N-1]
for k in range(N-2,-1,-1):
    ss = 0.0
    for j in range(k+1,N):
        ss = ss + A[k,j]*x[j]
    x[k] = (b[k] - ss)/A[k,k]

print("Solusi x:")
show_matrix(x)

Solusi x:


\begin{bmatrix}
  1.\\
  2.\\
  1.\\
\end{bmatrix}

Sekarang kita dapat mengecek apakah solusi yang kita dapatkan sudah benar.

In [94]:
show_matrix(A_orig)

\begin{bmatrix}
  1. & 1. & 1.\\
  2. & 3. & 1.\\
  1. & -1. & -1.\\
\end{bmatrix}

In [96]:
show_matrix(A_orig*x)

\begin{bmatrix}
  4.\\
  9.\\
  -2.\\
\end{bmatrix}

In [97]:
show_matrix(b_orig)

\begin{bmatrix}
  4.\\
  9.\\
  -2.\\
\end{bmatrix}

In [98]:
# Gunakan matrix dan vektor original
# Hasil seharunya berupa vektor kolom dengan elemen-element 0
show_matrix( A_orig*x - b_orig )

\begin{bmatrix}
  0.\\
  0.\\
  0.\\
\end{bmatrix}

### Kode Python untuk eliminasi Gauss

Berikut ini adalah implementasi metode eliminasi Gauss pada Python.

Kode ini menerima masukan matriks `A_` dan vektor kolom `b_`. dan mengembalikan solusi `x` dari sistem persamaan linear `A_*x = b_`. Tanda `_` digunakan untuk menunjukkan input asli. Pada kode berikut kita bekerja dengan matriks `A` dan vektor `x` yang merupakan kopi dari input-input awal.
Kode ini terbatas pada vektor input `b_` yang hanya terdiri dari satu kolom. Kode dapat dikembangkan untuk kasus kolom lebih dari satu. 

In [158]:
def gauss_elim(A_, b_):
    
    N, Nrhs = b_.shape
    
    assert Nrhs == 1

    A = np.matrix.copy(A_)
    b = np.matrix.copy(b_)

    # Eliminasi maju
    for k in range(0,N-1):
        for i in range(k+1,N):
            if A[i,k] != 0.0:
                alpha = A[i,k]/A[k,k]
                A[i,:] = A[i,:] - alpha*A[k,:]
                b[i] = b[i] - alpha*b[k]
    
    # Alokasi mememori untuk solusi
    x = np.matrix(np.zeros((N,1)))
    
    # Substitusi balik
    x[N-1] = b[N-1]/A[N-1,N-1]
    for k in range(N-2,-1,-1):
        ss = 0.0
        for j in range(k+1,N):
            ss = ss + A[k,j]*x[j]
        x[k] = (b[k] - ss)/A[k,k]
    
    return x

In [159]:
show_matrix( gauss_elim(A_orig, b_orig) )

\begin{bmatrix}
  1.\\
  2.\\
  1.\\
\end{bmatrix}

# Dekomposisi LU

Pada dekomposisi LU, matriks persegi $\mathbf{A}$ dapat dinyatakan sebagai hasil kali dari matriks segitiga bawah $\mathbf{L}$ dan matriks segitiga atas $\mathbf{U}$:
$$
\mathbf{A} = \mathbf{LU}
$$
Proses untuk mendapatkan matriks $\mathbf{L}$ dan $\mathbf{U}$ dari matriks $\mathbf{A}$ disebut dengan dekomposisi LU atau faktorisasi LU.
Dekomposisi LU tidak unik, artinya bisa terdapat banyak kombinasi $\mathbf{L}$ dan $\mathbf{U}$ yang mungkin untuk suatu matriks $\mathbf{A}$ yang diberikan.
Untuk mendapatkan pasangan $\mathbf{L}$ dan $\mathbf{U}$ yang unik, kita perlu memberikan batasan atau konstrain terhadap proses dekomposisi LU. Terdapat beberapa varian dekomposisi LU:

| Nama | Konstrain |
| ---- | --------- |
| Dekomposisi Doolittle | $L_{ii} = 1$ |
| Dekomposisi Crout | $U_{ii} = 1$ |
| Dekomposisi Cholesky | $\mathbf{L} = \mathbf{U}^{T}$ (untuk matriks $\mathbf{A}$ simetrik dan definit positif) |

Dengan dekomposisi LU kita dapat menuliskan sistem persamaan linear
$$
\mathbf{Ax} = \mathbf{b}
$$
menjadi:
$$
\mathbf{LUx} = \mathbf{b}
$$
Dengan menggunakan $\mathbf{y} = \mathbf{Ux}$ kita dapat menuliskan:
$$
\mathbf{Ly} = \mathbf{b}
$$
Persamaan ini dapat diselesaikan dengan menggunakan substitusi maju.
Solusi $\mathbf{x}$ dapat dicari dengan menggunakan substitusi balik (mundur).

Varian Doolittle untuk dekomposisi LU memiliki bentuk berikut untuk matriks $\mathbf{L}$ dan $\mathbf{U}$, misalnya untuk ukuran $3\times3$ 

$$
\mathbf{L} =
\begin{bmatrix}
1 & 0 & 0 \\
L_{21} & 1 & 0 \\
L_{31} & L_{32} & 1
\end{bmatrix},\,\,\,
\mathbf{U} =
\begin{bmatrix}
U_{11} & U_{12} & U_{13} \\
0 & U_{22} & U_{23} \\
0 & 0 & U_{33}
\end{bmatrix}
$$

Dengan melakukan perkalian $\mathbf{A} = \mathbf{LU}$

$$
\mathbf{A} = \begin{bmatrix}
U_{11} & U_{12} & U_{13} \\
U_{11}L_{21} & U_{12}L_{21} + U_{22} & U_{13}L_{21} + U_{23} \\
U_{11}L_{31} & U_{12}L_{31} + U_{22}L_{32} & U_{13}L_{31} + U_{23}L_{32} + U_{33}
\end{bmatrix}
$$

Dapat ditunjukkan bahwa matriks $\mathbf{U}$ adalah matriks segitiga atas hasil dari eliminasi Gauss pada matriks $\mathbf{A}$. Elemen *off-diagonal* dari matriks $\mathbf{L}$ adalah pengali $\alpha$, atau $L_{ij}$ adalah pengali yang mengelimisasi $A_{ij}$.

## Kode Python untuk dekomposisi LU (varian Doolittle)

In [160]:
def LU_decomp(A_):
    
    Nrow, Ncol = A_.shape
    
    assert Nrow == Ncol

    A = np.matrix.copy(A_)

    # Eliminasi Gauss (maju)
    for k in range(0,N-1):
        for i in range(k+1,N):
            if A[i,k] != 0.0:
                alpha = A[i,k]/A[k,k]
                A[i,k+1:N] = A[i,k+1:N] - alpha*A[k,k+1:N]
                A[i,k] = alpha
    
    L = np.matrix( np.tril(A,-1) )
    for i in range(N):
        L[i,i] = 1.0 # konstrain Doolittle
    U = np.matrix( np.triu(A) )
    
    return L, U # kembalikan matriks L dan U

Definisikan lagi matriks dan vektor yang ada pada contoh sebelumnya.

In [161]:
A = np.matrix([
    [1, 1, 1],
    [2, 3, 1],
    [1, -1, -1]
])
b = np.matrix([4, 9, -2]).transpose()

Lakukan dekomposisi LU:

In [167]:
L, U = LU_decomp(A)
print("L = "); show_matrix(L)
print("U = "); show_matrix(U)

L = 


\begin{bmatrix}
  1 & 0 & 0\\
  2 & 1 & 0\\
  1 & -2 & 1\\
\end{bmatrix}

U = 


\begin{bmatrix}
  1 & 1 & 1\\
  0 & 1 & -1\\
  0 & 0 & -4\\
\end{bmatrix}

Periksa bahwa $\mathbf{A} = \mathbf{LU}$:

In [168]:
show_matrix( L*U - A )

\begin{bmatrix}
  0 & 0 & 0\\
  0 & 0 & 0\\
  0 & 0 & 0\\
\end{bmatrix}

## Implementasi solusi dengan matrix L dan U yang sudah dihitung

Substitusi maju
$$
y_{k} = b_{k} - \sum_{j=1}^{k-1} L_{kj} y_{j},\,\,\,k = 2,3,\ldots,N
$$

In [169]:
def LU_solve(L, U, b):
    
    N = L.shape[0]
    
    x = np.matrix(np.zeros((N,))).transpose()
    y = np.matrix(np.zeros((N,))).transpose()
    
    # Ly = b
    # Substitusi maju
    y[0] = b[0]/L[0,0]
    for k in range(1,N):
        ss = 0.0
        for j in range(k):
            ss = ss + L[k,j]*y[j]
        y[k] = (b[k] - ss)/L[k,k]
    
    # Ux = y
    # Substitusi balik
    x[N-1] = y[N-1]/U[N-1,N-1]
    for k in range(N-2,-1,-1):
        ss = 0.0
        for j in range(k+1,N):
            ss = ss + U[k,j]*x[j]
        x[k] = (y[k] - ss)/U[k,k]
    
    return x

In [170]:
x = LU_solve(L, U, b)
show_matrix(x)

\begin{bmatrix}
  1.\\
  2.\\
  1.\\
\end{bmatrix}

Cek hasil:

In [171]:
show_matrix( A*x - b )

\begin{bmatrix}
  0.\\
  0.\\
  0.\\
\end{bmatrix}

# Latihan

## Latihan 1

Selasaikan persamaan:

$$
\begin{bmatrix}
6 & -4 & 1 \\
4 & 6 & -4 \\
1 & -4 & 6
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix} = 
\begin{bmatrix}
-14 \\
36 \\
6
\end{bmatrix}
$$

## Latihan 1 (Solusi)

Menggunakan eliminasi Gauss:

In [172]:
A = np.matrix([
    [6, -4, 1],
    [-4, 6, -4],
    [1, -4, 6]], dtype=np.float64)
b = np.matrix([-14, 36, 6], dtype=np.float64).transpose()
x = gauss_elim(A, b)
print("Solusi x = "); show_matrix(x)
print("Cek solusi: Ax - b = "); show_matrix(A*x - b)

Solusi x = 


\begin{bmatrix}
  10.\\
  22.\\
  14.\\
\end{bmatrix}

Cek solusi: Ax - b = 


\begin{bmatrix}
  1.77635684e-15\\
  2.13162821e-14\\
  0.00000000e+00\\
\end{bmatrix}

Menggunakan dekomposisi LU:

In [173]:
A = np.matrix([
    [6, -4, 1],
    [-4, 6, -4],
    [1, -4, 6]], dtype=np.float64)
b = np.matrix([-14, 36, 6], dtype=np.float64).transpose()
L, U = LU_decomp(A)
x = LU_solve(L, U, b)
print("Solusi x = "); show_matrix(x)
print("Cek solusi: Ax - b = "); show_matrix(A*x - b)

Solusi x = 


\begin{bmatrix}
  10.\\
  22.\\
  14.\\
\end{bmatrix}

Cek solusi: Ax - b = 


\begin{bmatrix}
  1.77635684e-15\\
  2.13162821e-14\\
  0.00000000e+00\\
\end{bmatrix}

## Latihan 2

Selasaikan persamaan:

$$
\begin{bmatrix}
3 & 1 & -1 \\
5 & 8 & 2 \\
3 & 1 & 10
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix} =
\begin{bmatrix}
1 \\
-4 \\
5
\end{bmatrix}
$$

Menggunakan eliminasi Gauss:

In [174]:
A = np.matrix( [
    [3, 1, -1],
    [5, 8, 2],
    [3, 1, 10]
], dtype=np.float64)
b = np.matrix([1, -4, 5], np.float64).transpose()
x = gauss_elim(A, b)
print("Solusi x = "); show_matrix(x)
print("Cek solusi: Ax - b = "); show_matrix(A*x - b)

Solusi x = 


\begin{bmatrix}
  0.82296651\\
  -1.10526316\\
  0.36363636\\
\end{bmatrix}

Cek solusi: Ax - b = 


\begin{bmatrix}
  -1.11022302e-16\\
  -8.88178420e-16\\
  0.00000000e+00\\
\end{bmatrix}

Menggunakan dekomposisi LU:

In [175]:
A = np.matrix( [
    [3, 1, -1],
    [5, 8, 2],
    [3, 1, 10]
], dtype=np.float64)
b = np.matrix([1, -4, 5], np.float64).transpose()
L, U = LU_decomp(A)
x = LU_solve(L, U, b)
print("Solusi x = "); show_matrix(x)
print("Cek solusi: Ax - b = "); show_matrix(A*x - b)

Solusi x = 


\begin{bmatrix}
  0.82296651\\
  -1.10526316\\
  0.36363636\\
\end{bmatrix}

Cek solusi: Ax - b = 


\begin{bmatrix}
  -1.11022302e-16\\
  -8.88178420e-16\\
  0.00000000e+00\\
\end{bmatrix}

# Pivoting

Metode eliminasi Gauss dan LU memiliki beberapa kekurangan. Salah satu yang sering ditemui adalah ketika elemen pivot yang ditemukan adalah 0. Misalkan pada persamaan berikut ini:

$$
\begin{bmatrix}
0 & -3 & 7 \\
1 & 2 & -1 \\
5 & -2 & 0
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix} = 
\begin{bmatrix}
2 \\
3 \\
4
\end{bmatrix}
$$

Jika kita langsung menggunakan fungsi `gauss_elim` kita akan mendapatkan pesan peringatan sebagai berikut:
```python
A = np.matrix( [
    [0, -3, 7],
    [1, 2, -1],
    [5, -2, 0]
], dtype=np.float64)
b = np.matrix([2, 3, 4], np.float64).transpose()
x = gauss_elim(A, b)
```

```
/home/efefer/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/home/efefer/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
/home/efefer/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
```

Solusi yang dapat digunakan adalah dengan cara menukar baris atau pivoting sedemikian rupa sehingga elemen pivot yang diperoleh tidak menjadi nol. Dalam kasus ini, kita dapat menukar baris pertama dengan baris ketiga:

$$
\begin{bmatrix}
5 & -2 & 0 \\
1 & 2 & -1 \\
0 & -3 & 7
\end{bmatrix}
\begin{bmatrix}
x_3 \\
x_2 \\
x_1
\end{bmatrix} =
\begin{bmatrix}
4 \\
3 \\
2
\end{bmatrix}
$$

In [176]:
A = np.matrix( [
    [5, -2, 0],
    [1, 2, -1],
    [0, -3, 7]
], dtype=np.float64)
b = np.matrix([4, 3, 2], np.float64).transpose()
x = gauss_elim(A, b)
print("Solusi x = "); show_matrix(x)
print("Cek solusi: Ax - b = "); show_matrix(A*x - b)

Solusi x = 


\begin{bmatrix}
  1.30434783\\
  1.26086957\\
  0.82608696\\
\end{bmatrix}

Cek solusi: Ax - b = 


\begin{bmatrix}
  0.0000000e+00\\
  4.4408921e-16\\
  -8.8817842e-16\\
\end{bmatrix}

Dapat dilihat bahwa pivoting pada dasarnya bertujuan untuk memperbaiki sifat dominan diagonal dari matriks. Suatu matriks dikatakan dominan diagonal apabila nilai absolut dari elemen diagoanal matriks memiliki nilai yang terbesar bila dibandingkan dengan nilai-nilai elemen lainnya dalam satu baris.

Ada beberapa strategi yang dapat digunakan untuk pivoting, salah satu yang paling sederhana adalah dengan menggunakan pivoting terskala. Dengan metode ini, pertama kali kita mencari array $\mathbf{s}$ dengan elemen sebagai berikut:

$$
s_{i} = \max\left|A_{ij}\right|,\,\,\,i=1,2,\ldots,N
$$

$s_{i}$ akan disebut sebagai faktor skala dari baris ke-$i$ yang merupakan elemen dengan nilai absolut terbesar dari baris ke-$i$.
Ukuran relatif dari elemen $A_{ij}$ relatif terhadap elemen dengan nilai terbesar adalah:
$$
r_{ij} = \frac{\left|A_{ij}\right|}{s_{i}}
$$
Elemen pivot dari matriks $\mathbf{A}$ akan ditentukan berdasarkan $r_{ij}$. Elemen $A_{kk}$ tidak secara otomatis menjadi elemen pivot, namun kita akan mencari elemen lain di bawah $A_{kk}$ pada kolom ke-$k$ untuk elemen pivot yang terbaik. Misalkan elemen terbaik ini ada pada baris ke-$p$, yaitu $A_{pk}$ yang memiliki ukuran relatif terbesar, yakni:
$$
r_{pk} = \max_{j} r_{jk},\,\,\, j \geq k
$$
Jika elemen tersebut ditemukan maka kita melakukan pertukaran baris antara baris ke-$k$ dan ke-$p$.

## Kode Python untuk eliminasi Gauss dengan pivoting

In [177]:
def tukar_baris(v, i, j):
    if len(v.shape) == 1: # array satu dimensi atau vektor kolom
        v[i], v[j] = v[j], v[i]
    else:
        v[[i,j],:] = v[[j,i],:]

In [178]:
def gauss_elim_pivot(A_, b_):
    N, Nrhs = b_.shape
    
    assert Nrhs == 1

    A = np.matrix.copy(A_)
    b = np.matrix.copy(b_)
    
    # Faktor skala
    s = np.matrix(np.zeros((N,1)))
    for i in range(N):
        s[i] = np.max(np.abs(A[i,:]))

    SMALL = np.finfo(np.float64).eps
    
    # Eliminasi maju
    for k in range(0,N-1):
        
        r = np.abs(A[k:N,k])/s[k:N]
        p = np.argmax(r) + k
        if abs(A[p,k]) < SMALL:
            raise RuntimeError("Matriks A singular")
        # Tukar baris jika diperlukan
        if p != k:
            print("INFO: tukar baris %d dengan %d" % (p,k))
            tukar_baris(b, k, p)
            tukar_baris(s, k, p)
            tukar_baris(A, k, p)
        
        for i in range(k+1,N):
            if A[i,k] != 0.0:
                alpha = A[i,k]/A[k,k]
                A[i,:] = A[i,:] - alpha*A[k,:]
                b[i] = b[i] - alpha*b[k]
    
    # Alokasi mememori untuk solusi
    x = np.matrix(np.zeros((N,1)))
    
    # Substitusi balik
    x[N-1] = b[N-1]/A[N-1,N-1]
    for k in range(N-2,-1,-1):
        ss = 0.0
        for j in range(k+1,N):
            ss = ss + A[k,j]*x[j]
        x[k] = (b[k] - ss)/A[k,k]
    
    return x

Contoh penggunaan:

In [179]:
A = np.matrix( [
    [0, -3, 7],
    [1, 2, -1],
    [5, -2, 0]
], dtype=np.float64)
b = np.matrix([2, 3, 4], np.float64).transpose()

x = gauss_elim_pivot(A, b)
print("Solusi x = "); show_matrix(x)
print("Cek solusi: Ax - b = "); show_matrix(A*x - b)

INFO: tukar baris 2 dengan 0
Solusi x = 


\begin{bmatrix}
  1.30434783\\
  1.26086957\\
  0.82608696\\
\end{bmatrix}

Cek solusi: Ax - b = 


\begin{bmatrix}
  -8.8817842e-16\\
  4.4408921e-16\\
  0.0000000e+00\\
\end{bmatrix}

## Kode Python untuk dekomposisi LU dengan pivoting

In [180]:
def LU_decomp_pivot(A_):
    
    Nrow, Ncol = A_.shape
    
    assert Nrow == Ncol
    
    N = Nrow

    A = np.matrix.copy(A_)
    
    # Faktor skala
    s = np.matrix(np.zeros((N,1)))
    for i in range(N):
        s[i] = np.max(np.abs(A[i,:]))
        
    iperm = np.arange(N)

    SMALL = np.finfo(np.float64).eps

    # Eliminasi Gauss (maju)
    for k in range(0,N-1):
        
        r = np.abs(A[k:N,k])/s[k:N]
        p = np.argmax(r) + k
        if abs(A[p,k]) < SMALL:
            raise RuntimeError("Matriks A singular")
        # Tukar baris jika diperlukan
        if p != k:
            print("INFO: tukar baris %d dengan %d" % (p,k))
            tukar_baris(A, k, p)
            tukar_baris(s, k, p)
            tukar_baris(iperm, k, p)
        
        for i in range(k+1,N):
            if A[i,k] != 0.0:
                alpha = A[i,k]/A[k,k]
                A[i,k+1:N] = A[i,k+1:N] - alpha*A[k,k+1:N]
                A[i,k] = alpha
    
    L = np.matrix( np.tril(A,-1) )
    for i in range(N):
        L[i,i] = 1.0 # konstrain Doolittle
    U = np.matrix( np.triu(A) )
    
    return L, U, iperm # kembalikan matriks L dan U serta vektor permutasi

In [181]:
def LU_solve_pivot(L, U, iperm, b_):
    
    N = L.shape[0]
    
    x = np.matrix(np.zeros((N,))).transpose()
    y = np.matrix(np.zeros((N,))).transpose()
    
    b = np.matrix.copy(b_)
    for i in range(N):
        b[i] = b_[iperm[i]]
    
    # Ly = b
    # Substitusi maju
    y[0] = b[0]/L[0,0]
    for k in range(1,N):
        ss = 0.0
        for j in range(k):
            ss = ss + L[k,j]*y[j]
        y[k] = (b[k] - ss)/L[k,k]
    
    # Ux = y
    # Substitusi balik
    x[N-1] = y[N-1]/U[N-1,N-1]
    for k in range(N-2,-1,-1):
        ss = 0.0
        for j in range(k+1,N):
            ss = ss + U[k,j]*x[j]
        x[k] = (y[k] - ss)/U[k,k]
    
    return x

Contoh penggunaan:

In [184]:
A = np.matrix([
    [2, -2, 6],
    [-2, 4, 3],
    [-1, 8, 4]
], dtype=np.float64)
b = np.matrix([16, 0, -1]).transpose()
L, U, iperm = LU_decomp_pivot(A)
print("L = "); show_matrix(L)
print("U = "); show_matrix(U)
x = LU_solve_pivot(L, U, iperm, b)
print("Solusi x = "); show_matrix(x)
print("Cek solusi: Ax - b = "); show_matrix(A*x - b)

INFO: tukar baris 1 dengan 0
INFO: tukar baris 2 dengan 1
L = 


\begin{bmatrix}
  1. & 0. & 0.\\
  0.5 & 1. & 0.\\
  -1. & 0.33333333 & 1.\\
\end{bmatrix}

U = 


\begin{bmatrix}
  -2. & 4. & 3.\\
  0. & 6. & 2.5\\
  0. & 0. & 8.16666667\\
\end{bmatrix}

Solusi x = 


\begin{bmatrix}
  1.\\
  -1.\\
  2.\\
\end{bmatrix}

Cek solusi: Ax - b = 


\begin{bmatrix}
  0.\\
  0.\\
  0.\\
\end{bmatrix}

Contoh lain:

In [185]:
A = np.matrix([
    [0, 2, 5, -1],
    [2, 1, 3, 0],
    [-2, -1, 3, 1],
    [3, 3, -1, 2]
], dtype=np.float64)
b = np.matrix([-3, 3, -2, 5]).transpose()
L, U, iperm = LU_decomp_pivot(A)
print("L = "); show_matrix(L)
print("U = "); show_matrix(U)
x = LU_solve_pivot(L, U, iperm, b)
print("Solusi x = "); show_matrix(x)
print("Cek solusi: Ax - b = "); show_matrix(A*x - b)

INFO: tukar baris 3 dengan 0
INFO: tukar baris 3 dengan 1
INFO: tukar baris 3 dengan 2
L = 


\begin{bmatrix}
  1. & 0. & 0. & 0.\\
  0. & 1. & 0. & 0.\\
  0.66666667 & -0.5 & 1. & 0.\\
  -0.66666667 & 0.5 & -0.02702703 & 1.\\
\end{bmatrix}

U = 


\begin{bmatrix}
  3. & 3. & -1. & 2.\\
  0. & 2. & 5. & -1.\\
  0. & 0. & 6.16666667 & -1.83333333\\
  0. & 0. & 0. & 2.78378378\\
\end{bmatrix}

Solusi x = 


\begin{bmatrix}
  2.00000000e+00\\
  -1.00000000e+00\\
  3.60072332e-17\\
  1.00000000e+00\\
\end{bmatrix}

Cek solusi: Ax - b = 


\begin{bmatrix}
  0.\\
  0.\\
  0.\\
  0.\\
\end{bmatrix}

# Menggunakan pustaka pada Python

Untuk berbagai aplikasi pada sains dan teknik, kita biasanya menyelesaikan sistem persamaan linear yang ditemui dengan menggunakan berbagai macam pustaka yang sudah tersedia.

Python sudah memiliki beberapa fungsi yang terkait dengan sistem persamaan linear dan operasi terkait seperti menghitung determinan dan invers matriks.

Fungsi `np.linalg.solve` dapat digunakan untuk menyelesaikan sistem persamaan linear:

In [186]:
A = np.matrix([
    [1, 1, 1],
    [2, 3, -1],
    [1, -1, -1]
])
B = np.matrix([4, 9, 2]).transpose()

In [187]:
x = np.linalg.solve(A,B)
show_matrix(x)

\begin{bmatrix}
  3.\\
  1.\\
  0.\\
\end{bmatrix}

In [188]:
show_matrix(A*x - B)

\begin{bmatrix}
  0.\\
  0.\\
  0.\\
\end{bmatrix}

Fungsi `np.linalg.det` dapat digunakan untuk menghitung determinan dari suatu matriks

In [189]:
np.linalg.det(A)

-8.000000000000002

Fungsi `np.linalg.inv` dapat digunakan untuk menghitung invers dari suatu matriks

In [191]:
show_matrix( np.linalg.inv(A) )

\begin{bmatrix}
  0.5 & 0. & 0.5\\
  -0.125 & 0.25 & -0.375\\
  0.625 & -0.25 & -0.125\\
\end{bmatrix}